## Ridge Regression

In [131]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import scale
from math import sqrt

In [132]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
# load dataset

dataframe = pd.read_csv("aafc_data.csv")
# summarize shape
print(dataframe.shape)
print("\n")
# summarize first few lines
dataframe.head()

(42007, 166)




,Unnamed: 0,TWP_ID,ECODISTRICT_ID,YEAR,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,0,00101E1,852.0,2010,867.766846,53.6,111.1,109.7,117.9,46.4,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,1,00101W1,852.0,2010,673.685028,57.2,114.7,110.5,114.0,46.2,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,2,00101W2,796.0,2010,824.303864,39.0,96.4,109.8,101.2,111.4,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,3,00102E1,853.0,2010,1006.708496,37.5,158.2,157.8,161.4,46.9,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,4,00102W1,852.0,2010,869.040283,57.2,114.7,110.5,114.0,46.2,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


In [133]:
dataframe.drop(['Unnamed: 0', 'TWP_ID', 'YEAR'], axis=1, inplace=True)

In [134]:
dataframe.head()

,ECODISTRICT_ID,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,SumPcpn23_25,SumPcpn24_26,SumPcpn25_27,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,852.0,867.766846,53.6,111.1,109.7,117.9,46.4,69.3,60.0,44.6,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,852.0,673.685028,57.2,114.7,110.5,114.0,46.2,68.1,55.9,34.9,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,796.0,824.303864,39.0,96.4,109.8,101.2,111.4,153.0,163.6,98.8,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,853.0,1006.708496,37.5,158.2,157.8,161.4,46.9,79.5,67.5,40.4,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,852.0,869.040283,57.2,114.7,110.5,114.0,46.2,68.1,55.9,34.9,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


---

In [135]:
df1 = dataframe[dataframe['ECODISTRICT_ID']==748]
df1.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df1.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df1['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [136]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)

print(X_train.shape); print(X_test.shape)


(839, 161)
(360, 161)


In [137]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  76.1919905362895
R squared training set:  84.72096804187296
Test MSE:  87.90676524170678
R squared test set:  80.78561045557035
Mean Absolute Error: YieldKgAcre    69.988648
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.11
dtype: float64 %.


---

In [138]:
df2 = dataframe[dataframe['ECODISTRICT_ID']==826]
df2.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df2.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df2['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [139]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)

print(X_train.shape); print(X_test.shape)

(823, 161)
(354, 161)


In [140]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  101.09165827651603
R squared training set:  75.60317443634008
Test MSE:  105.57282197153029
R squared test set:  73.57893809551786
Mean Absolute Error: YieldKgAcre    82.970694
dtype: float64 degrees.
Accuracy: YieldKgAcre    87.34
dtype: float64 %.


---

In [141]:
df3 = dataframe[dataframe['ECODISTRICT_ID']==752]
df3.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df3.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df3['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [142]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)

print(X_train.shape); print(X_test.shape)

(777, 161)
(334, 161)


In [143]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  78.66255663549335
R squared training set:  86.53688382863933
Test MSE:  92.89526127482065
R squared test set:  77.86303456647643
Mean Absolute Error: YieldKgAcre    71.969368
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.07
dtype: float64 %.


---

In [144]:
df4 = dataframe[dataframe['ECODISTRICT_ID']==745]
df4.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df4.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df4['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [145]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)


print(X_train.shape); print(X_test.shape)

(770, 161)
(330, 161)


In [146]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  74.30969520931816
R squared training set:  89.2187411341816
Test MSE:  83.60354856900985
R squared test set:  87.42100132377485
Mean Absolute Error: YieldKgAcre    64.359002
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.73
dtype: float64 %.


---

In [147]:
df5 = dataframe[dataframe['ECODISTRICT_ID']==808]
df5.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df5.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df5['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [148]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   
   
# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)     
        
        
print(X_train.shape); print(X_test.shape)

(746, 161)
(321, 161)


In [149]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  108.80776688895307
R squared training set:  71.47017415604348
Test MSE:  125.33205832535845
R squared test set:  60.32007296901117
Mean Absolute Error: YieldKgAcre    93.876136
dtype: float64 degrees.
Accuracy: YieldKgAcre    87.32
dtype: float64 %.


---

In [150]:
df6 = dataframe[dataframe['ECODISTRICT_ID']==792]
df6.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df6.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df6['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [151]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)  

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)
  
    
print(X_train.shape); print(X_test.shape)

(708, 161)
(304, 161)


In [152]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  95.13732923966857
R squared training set:  78.53762506583996
Test MSE:  107.29538676736196
R squared test set:  72.22176207981319
Mean Absolute Error: YieldKgAcre    81.675159
dtype: float64 degrees.
Accuracy: YieldKgAcre    89.82
dtype: float64 %.


---

In [153]:
df7 = dataframe[dataframe['ECODISTRICT_ID']==849]
df7.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df7.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df7['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [154]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   
  
# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)     
        
        
print(X_train.shape); print(X_test.shape)

(700, 161)
(301, 161)


In [155]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  78.54796667665148
R squared training set:  90.73648086893138
Test MSE:  95.63859918528432
R squared test set:  85.97108872906384
Mean Absolute Error: YieldKgAcre    71.924404
dtype: float64 degrees.
Accuracy: YieldKgAcre    89.99
dtype: float64 %.


---

In [156]:
df8 = dataframe[dataframe['ECODISTRICT_ID']==729]
df8.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df8.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df8['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [157]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test) 
     
         
print(X_train.shape); print(X_test.shape)

(662, 161)
(284, 161)


In [158]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  71.19193192790989
R squared training set:  81.91779666344638
Test MSE:  98.91196333229948
R squared test set:  67.7959532281787
Mean Absolute Error: YieldKgAcre    75.362232
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.5
dtype: float64 %.


---

In [159]:
df9 = dataframe[dataframe['ECODISTRICT_ID']==753]
df9.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df9.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df9['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [160]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   
  
# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)  

print(X_train.shape); print(X_test.shape)

(662, 161)
(284, 161)


In [161]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  58.01717143917308
R squared training set:  89.16091262673463
Test MSE:  70.3129400188808
R squared test set:  84.31207850205715
Mean Absolute Error: YieldKgAcre    56.129276
dtype: float64 degrees.
Accuracy: YieldKgAcre    93.31
dtype: float64 %.


---

In [162]:
df10 = dataframe[dataframe['ECODISTRICT_ID']==709]
df10.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df10.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df10['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [163]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)
     
print(X_train.shape); print(X_test.shape)

(639, 161)
(274, 161)


In [164]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  84.4591216123592
R squared training set:  87.89976256831535
Test MSE:  104.67585292319163
R squared test set:  82.28193554839056
Mean Absolute Error: YieldKgAcre    80.437732
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.28
dtype: float64 %.
